In [1]:
import data
import torch
from models import imagebind_model
from models.imagebind_model import ModalityType

import pickle, os, logging
import numpy as np

/home/iosz/.conda/envs/sen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/iosz/.conda/envs/sen/lib/python3.8/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/home/iosz/.conda/envs/sen/lib/python3.8/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/iosz/.conda/envs/sen/lib/python3.8/site-p

In [6]:
text_list=["A dog", "A car", "A bird"]
image_paths=[".assets/dog_image.jpg", ".assets/car_image.jpg", ".assets/bird_image.jpg"]
audio_paths=[".assets/dog_audio.wav", ".assets/car_audio.wav", ".assets/bird_audio.wav"]

In [7]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

# Instantiate model
model = imagebind_model.imagebind_huge(pretrained=True)
model.eval()
model.to(device)

print(device)

# # # Load data
# inputs = {
#     ModalityType.TEXT: data.load_and_transform_text(text_list, device),
#     ModalityType.VISION: data.load_and_transform_vision_data(image_paths, device),
#     ModalityType.AUDIO: data.load_and_transform_audio_data(audio_paths, device),
# }

# with torch.no_grad():
#     embeddings = model(inputs)

# print(
#     "Vision x Text: ",
#     torch.softmax(embeddings[ModalityType.VISION] @ embeddings[ModalityType.TEXT].T, dim=-1),
# )
# print(
#     "Audio x Text: ",
#     torch.softmax(embeddings[ModalityType.AUDIO] @ embeddings[ModalityType.TEXT].T, dim=-1),
# )
# print(
#     "Vision x Audio: ",
#     torch.softmax(embeddings[ModalityType.VISION] @ embeddings[ModalityType.AUDIO].T, dim=-1),
# )

cuda:0


### Dump the model to pickle

In [8]:
# model = model.cpu()

# with open('imagebind.pickle', 'wb') as handle:
#     pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Select target labels

In [9]:
TARGETS = [
    "Acoustic_guitar",
    "Applause",
    "Bark",
    "Bass_drum",
    "Burping_or_eructation",
    "Bus",
    "Cello",
    "Chime",
    "Clarinet",
    "Computer_keyboard",
    "Cough",
    "Cowbell",
    "Double_bass",
    "Drawer_open_or_close",
    "Electric_piano",
    "Fart",
    "Finger_snapping",
    "Fireworks",
    "Flute",
    "Glockenspiel",
    "Gong",
    "Gunshot_or_gunfire",
    "Harmonica",
    "Hi-hat",
    "Keys_jangling",
    "Knock",
    "Laughter",
    "Meow",
    "Microwave_oven",
    "Oboe",
    "Saxophone",
    "Scissors",
    "Shatter",
    "Snare_drum",
    "Squeak",
    "Tambourine",
    "Tearing",
    "Telephone",
    "Trumpet",
    "Violin_or_fiddle",
    "Writing",
]

TARGETS_CLEAN = [
    "Acoustic guitar",
    "Applause",
    "Bark",
    "Bass drum",
    "Burping or eructation",
    "Bus",
    "Cello",
    "Chime",
    "Clarinet",
    "Computer keyboard",
    "Cough",
    "Cowbell",
    "Double bass",
    "Drawer open or close",
    "Electric piano",
    "Fart",
    "Finger snapping",
    "Fireworks",
    "Flute",
    "Glockenspiel",
    "Gong",
    "Gunshot or gunfire",
    "Harmonica",
    "Hi-hat",
    "Keys jangling",
    "Knock",
    "Laughter",
    "Meow",
    "Microwave oven",
    "Oboe",
    "Saxophone",
    "Scissors",
    "Shatter",
    "Snare drum",
    "Squeak",
    "Tambourine",
    "Tearing",
    "Telephone",
    "Trumpet",
    "Violin or fiddle",
    "Writing",
]

In [10]:
# # parse all labels and add to a dict
# inputs = { ModalityType.TEXT: data.load_and_transform_text(TARGETS_CLEAN, device) }

In [11]:
# # Get embeddings
# with torch.no_grad():
#     embeddings = model(inputs)
    
# # put embeddings on cpu
# #for key, val in ModalityType.__dict__.items():
# embeddings["text"] = np.array(embeddings["text"].to("cpu"))

In [12]:
# embeddings[ModalityType.TEXT][0].shape

In [13]:
# with open('emb-imagebind-orig.pickle', 'wb') as handle:
#     pickle.dump(embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:


# # Load precomputed embeddings
# with open('emb-imagebind-orig.pickle', 'rb') as handle:
#     embeddings = pickle.load(handle)

# embedding_mod = "text"
# embedding_len = len(embeddings["text"][0])  # get emb dim    

# # Construct the query embedding vector
# if len(args.targets) == 0:
#     # case with no target
#     query = torch.ones(1, len(embedding_len)) 
# elif len(args.targets) == 1:
#     # case with single target
#     query = embeddings[embedding_mod][TARGETS.index(t)]
# else:
#     # case with multiple targets
#     emb_list = []
#     for t in args.targets:
#         emb_list.append(embeddings[embedding_mod][TARGETS.index(t)])
    
#     # average the embeddings
#     query = np.mean(emb_list, axis=0)

In [15]:
arr = np.array([[1, 0, 0, 1], [0, 1, 1, 0]])

# print(arr)

for i, row in enumerate(arr):
    print(np.where(row==1)[0])

[0 3]
[1 2]


In [16]:
np.where(arr==1)

(array([0, 0, 1, 1]), array([0, 3, 1, 2]))

In [17]:
np.mean(arr[0])

0.5

In [18]:
arr

array([[1, 0, 0, 1],
       [0, 1, 1, 0]])

### Precompute embeddings from all audio targets

In [19]:
def get_files_in_folder(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

def compute_embeddings(embedding_dict, data_split, project_path):
    
    tensor_list = []
    # train data path
    for idx, target in enumerate(TARGETS):
        target_folder_path = os.path.join(project_path, "data", "FSDSoundScapes", "FSDKaggle2018", data_split, str(target))

        file_paths = get_files_in_folder(target_folder_path)  

        # Load data
        inputs = {
            modality: data.load_and_transform_audio_data(file_paths, device)
        }

        # calculate embeddings
        with torch.no_grad():
            embedding = model(inputs)
            
        tensor_list.append(embedding[ModalityType.AUDIO])
        
    # embedding_dict[data_split] = torch.stack(tensor_list, dim=0)
    
    embedding_dict[data_split] = tensor_list
    
        # embedding_dict[data_split] = embedding[ModalityType.AUDIO]

In [20]:
project_path = '/scratch/IOSZ/waveformer/multimod-sound-separation/multimod-waveformer'

#### The embeddings are stored in a two level dict of shape: 
data_split(train/eval/test) -> list(tensor(n_samples, enc_dim))

In [21]:
logger = logging.getLogger()

In [22]:
#embeddings_dict = { 'train': {}, 'val': {}, 'test': {} }  # define initial structure
embeddings_dict = {}

modality = 'audio'

In [23]:
# train data split
logger.setLevel(logging.ERROR)
compute_embeddings(embeddings_dict, data_split='train', project_path=project_path)  # pass by reference

In [24]:
# val data split
logger.setLevel(logging.ERROR)
compute_embeddings(embeddings_dict, data_split='val', project_path=project_path)  # pass by reference

In [25]:
# test data split
logger.setLevel(logging.ERROR)
compute_embeddings(embeddings_dict, data_split='test', project_path=project_path)  # pass by reference

In [29]:
with open('emb-imagebind-audio.pickle', 'wb') as handle:
    pickle.dump(embeddings_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
# if str.lower(mode) == 'test':
#     mode = 'eval'

# curr_label = self.TARGETS[index]  # current label string
# folder_path = os.path.join(os.getcwd(), "data", "FSDSoundScapes", "FSDKaggle2018", str.lower(mode), str(curr_label))
# audio_path = os.path.join(folder_path, random.choice(os.listdir(folder_path)))  # choose one of the files

# # print("AUDIO PATH: ", audio_path)

# # Load data
# inputs = {
#     modality: data_imagebind.load_and_transform_audio_data([audio_path], self.device)
# }

# # calculate embeddings
# with torch.no_grad():
#     embedding = self.embedding_model(inputs)

# return embedding[modality][0]

In [33]:
embeddings_dict['train'][0]

tensor([[ 0.4710, -0.3620, -0.0745,  ...,  0.8085,  0.3394, -0.2203],
        [-0.3242,  0.0505, -0.2391,  ...,  0.7286, -1.0235,  0.2464],
        [-0.2584, -0.0451, -0.4842,  ...,  0.2111, -0.4052, -0.1214],
        ...,
        [-0.5626,  0.1492,  0.0880,  ...,  0.9239, -0.8681, -0.2784],
        [-0.3582,  0.6139,  0.0278,  ...,  0.7972, -0.2738,  0.4377],
        [-0.3770,  0.4972,  0.1245,  ...,  0.7261, -0.2226, -0.3311]],
       device='cuda:0')

In [15]:
# with open('emb-imagebind-audio.pickle', 'rb') as f:
#     data = pickle.load(f)
tensor_list = [torch.tensor([1,2,3],dtype=torch.float32), torch.tensor([2,3,4],dtype=torch.float32)]

In [22]:
torch.stack(tensor_list, dim=0)

tensor([[1., 2., 3.],
        [2., 3., 4.]])

In [12]:
tensor = torch.tensor([[1,2,3], [4,5,6]], dtype=torch.float32)
tensor.shape

torch.Size([2, 3])

In [19]:
torch.mean(torch.stack(tensor_list, dim=0), axis=0)

tensor([1.5000, 2.5000, 3.5000])